# WEB Scraping libraries

In [1]:
#pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [280]:
import requests
import pandas as pd
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import time
import random
from tqdm import tqdm

# Wikipedia headers

In [1]:
def display_wiki_headers(url):
    #url = 'https://en.wikipedia.org/wiki/Main_Page'
    r = requests.get (url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    for i in soup.find_all("h2"):
        print(i.text)

In [6]:
display_wiki_headers("https://en.wikipedia.org/wiki/Main_Page")

From today's featured article
Did you know ...
In the news
On this day
Today's featured picture
Other areas of Wikipedia
Wikipedia's sister projects
Wikipedia languages
Navigation menu


# IMDB Top 100 movies

In [56]:
def get_top_100_imdb_movies(url):
    #url = 'https://www.imdb.com/search/title/?groups=top_100&start=1'
    r = requests.get (url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    
    rating = []
    name = []
    year = []
    for i in soup.find_all("div",{"class":"inline-block ratings-imdb-rating"}):
        rating.append(i.find('strong').text)
    for i in soup.find_all("h3",{"class":"lister-item-header"}):
        name.append(i.find("a").text)
        year.append(i.find("span",{"class":"lister-item-year text-muted unbold"}).text.replace(")","").replace("(",""))
    df = pd.DataFrame({"Movie_Name":name,"Year":year,"Rating":rating})
    if len(df) < 100:
        url = url+"&start="+str(100-len(df)+1)
        r = requests.get (url)
        htmlContent = r.content
        soup = BeautifulSoup(htmlContent, 'html.parser')

        rating = []
        name = []
        year = []
        for i in soup.find_all("div",{"class":"inline-block ratings-imdb-rating"}):
            rating.append(i.find('strong').text)
        for i in soup.find_all("h3",{"class":"lister-item-header"}):
            name.append(i.find("a").text)
            year.append(i.find("span",{"class":"lister-item-year text-muted unbold"}).text.replace(")","").replace("(",""))
        df = df.append(pd.DataFrame({"Movie_Name":name,"Year":year,"Rating":rating}))
        
    df.to_csv("Imdb_top_100_movie_list.csv",index=False)
    
    
    print(df.reset_index())

In [57]:
get_top_100_imdb_movies("https://www.imdb.com/search/title/?groups=top_100")

    index                                         Movie_Name    Year Rating
0       0                                  Avengers: Endgame    2019    8.4
1       1                                       Gisaengchung    2019    8.6
2       2                           The Shawshank Redemption    1994    9.3
3       3                                      The Godfather    1972    9.2
4       4                                              Joker    2019    8.4
5       5                                          Inception    2010    8.8
6       6                                    The Dark Knight    2008    9.0
7       7                                       Interstellar    2014    8.6
8       8                                          Star Wars    1977    8.6
9       9  The Lord of the Rings: The Fellowship of the Ring    2001    8.8
10     10                           The Silence of the Lambs    1991    8.6
11     11                                 Back to the Future    1985    8.5
12     12   

# IMDB Top 100 Indian Movies

In [72]:
def get_imdb_top_100_indian_movies(url):
    #url = 'https://www.imdb.com/india/top-rated-indian-movies/'
    r = requests.get (url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    rating = []
    name = []
    year = []
    for j in soup.find_all("td",{"class":"ratingColumn imdbRating"}):
        rating.append(j.find("strong").text)
    for i in soup.find_all("td",{"class":"titleColumn"}):
        name.append(i.find("a").text)
        year.append(i.find("span",{"class":"secondaryInfo"}).text.replace(")","").replace("(",""))
    df = pd.DataFrame({"Movie_Name":name,"Year":year,"Rating":rating})
    df.iloc[:100].to_csv("Imdb_top_100_indian_movie_list.csv",index=False)
    print(df.iloc[:100])

In [73]:
get_imdb_top_100_indian_movies("https://www.imdb.com/india/top-rated-indian-movies/")

                           Movie_Name  Year Rating
0                     Pather Panchali  1955    8.5
1                             Nayakan  1987    8.5
2                            Gol Maal  1979    8.5
3                          Anbe Sivam  2003    8.5
4                         Apur Sansar  1959    8.5
5                   C/o Kancharapalem  2018    8.5
6                          Drishyam 2  2021    8.5
7                   Pariyerum Perumal  2018    8.5
8                            Kireedam  1989    8.4
9                           Natsamrat  2016    8.4
10                   Manichitrathazhu  1993    8.4
11                       Black Friday  2004    8.4
12                                 96  2018    8.4
13                       Thevar Magan  1992    8.4
14                  Kumbalangi Nights  2019    8.4
15                           3 Idiots  2009    8.3
16                         Visaaranai  2015    8.3
17                   Taare Zameen Par  2007    8.3
18                           Ra

# Bookpage.com

In [124]:
def get_book_info(url):
    #url = 'https://bookpage.com/reviews'
    r = requests.get(url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    name, author, genre, review = [], [], [], []
    for i in soup.find_all("div",{"class":"row-fluid article-row"}):
        name.append(i.find_all("a")[1].text.replace(" ★ ",""))
        author.append(i.find_all("p")[0].text.replace("\n",""))
        genre.append(i.find_all("a")[2].text)
        review_url = "https://bookpage.com"+i.find_all("a")[0]["href"]
        r1 = requests.get(review_url)
        htmlContent1 = r1.content
        soup1 = BeautifulSoup(htmlContent1, 'html.parser')
        review.append(soup1.find("div",{"class":"article-body"}).text.replace("\n"," ").replace("\xa0",""))
    df = pd.DataFrame({"Book_Name":name,"Author":author,"Genre":genre,"Review":review})
    df.iloc[:5].to_csv("Bookpage_book_list.csv",index=False)
    print(df.iloc[:5])

In [125]:
get_book_info('https://bookpage.com/reviews')

                   Book_Name                         Author       Genre  \
0  A Special Place for Women                   Laura Hankin     Fiction   
1                   Not Yeti  Kelly DiPucchio, Claire Keane  Children's   
2                   The Plot            Jean Hanff Korelitz     Fiction   
3             Notes on Grief       Chimamanda Ngozi Adichie  Nonfiction   
4         Out of the Shadows               Emily Midorikawa  Nonfiction   

                                              Review  
0    Laura Hankin’s A Special Place for Women has...  
1    A softhearted, cyan-colored creature, Yeti r...  
2    Not every 350-page novel can be torn through...  
3    What is the shape of grief? For writer Chima...  
4    Spirit mediums have been capturing imaginati...  


# ICC-Cricket.com

In [181]:
def get_team_ranking(url):
    r = requests.get(url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    team, matches, points, ratings = [], [], [], []
    team.append(soup.find("tbody").find("tr").find_all("span")[1].text)
    matches.append(soup.find("td",{"class":"rankings-block__banner--matches"}).text)
    points.append(soup.find("td",{"class":"rankings-block__banner--points"}).text.replace(",",""))
    ratings.append(soup.find("td",{"class":"rankings-block__banner--rating u-text-right"}).text.strip())
    
    for i in soup.find_all("tr",{"class":"table-body"}):
        team.append(i.find_all("span")[1].text)
    for i in soup.find_all("tr",{"class":"table-body"}):
        matches.append(i.find_all("td")[2].text)
        points.append(i.find_all("td")[3].text.replace(",",""))
        ratings.append(i.find_all("td")[4].text)
    
    df = pd.DataFrame({"TEAM":team,"MATCHES":matches,"POINTS":points,"RATING":ratings})
    
    print(df.iloc[:10]) 

## Women team ranking

In [182]:
get_team_ranking('https://www.icc-cricket.com/rankings/womens/team-rankings/odi')

           TEAM MATCHES POINTS RATING
0     Australia      18   2955    164
1  South Africa      24   2828    118
2       England      17   1993    117
3         India      20   2226    111
4   New Zealand      21   1947     93
5   West Indies      12   1025     85
6      Pakistan      15   1101     73
7    Bangladesh       5    306     61
8     Sri Lanka      11    519     47
9       Ireland       2     25     13


## Men team ranking

In [183]:
get_team_ranking("https://www.icc-cricket.com/rankings/mens/team-rankings/odi")

           TEAM MATCHES POINTS RATING
0   New Zealand      17   2054    121
1     Australia      25   2945    118
2         India      29   3344    115
3       England      27   3100    115
4  South Africa      20   2137    107
5      Pakistan      24   2323     97
6    Bangladesh      24   2157     90
7   West Indies      27   2222     82
8     Sri Lanka      21   1652     79
9   Afghanistan      17   1054     62


In [237]:
def get_player_ranking(url):
    #url = 'https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting'
    r = requests.get(url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    name, team, rating = [], [], []
    name.append(soup.find("tr",{"class":"rankings-block__banner"}).find_all("div")[3].text)
    team.append(soup.find("tr",{"class":"rankings-block__banner"}).find_all("div")[-2].text.strip())
    rating.append(soup.find("div",{"class":"rankings-block__banner--rating"}).text)
    
    for i in soup.find_all("tr",{"class":"table-body"}):
        name.append(i.find("td",{"class":"table-body__cell rankings-table__name name"}).find("a").text)
        team.append(i.find_all("span")[-1].text)
        rating.append(i.find_all("td")[-2].text)
    df = pd.DataFrame({"PLAYER":name,"TEAM":team,"RATING":rating})
    print(df.iloc[:10])

## Top 10 ODI Batsman Mens

In [239]:
get_player_ranking("https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting")

                PLAYER TEAM RATING
0           Babar Azam  PAK    865
1          Virat Kohli  IND    857
2         Rohit Sharma  IND    825
3          Ross Taylor   NZ    801
4          Aaron Finch  AUS    791
5       Jonny Bairstow  ENG    785
6         Fakhar Zaman  PAK    778
7  Francois du Plessis   SA    778
8         David Warner  AUS    773
9            Shai Hope   WI    773


## Top 10 ODI Bowler Mens

In [240]:
get_player_ranking("https://www.icc-cricket.com/rankings/mens/player-rankings/odi/bowling")

             PLAYER TEAM RATING
0       Trent Boult   NZ    737
1  Mujeeb Ur Rahman  AFG    708
2        Matt Henry   NZ    691
3    Jasprit Bumrah  IND    690
4      Mehedi Hasan  BAN    668
5     Kagiso Rabada   SA    666
6      Chris Woakes  ENG    665
7    Josh Hazlewood  AUS    660
8       Pat Cummins  AUS    646
9     Mohammad Amir  PAK    638


## Top 10 ODI All-rounder Womens

In [241]:
get_player_ranking("https://www.icc-cricket.com/rankings/womens/player-rankings/odi/all-rounder")

             PLAYER TEAM RATING
0    Marizanne Kapp   SA    418
1      Ellyse Perry  AUS    418
2   Stafanie Taylor   WI    410
3    Natalie Sciver  ENG    349
4     Deepti Sharma  IND    343
5     Jess Jonassen  AUS    307
6  Ashleigh Gardner  AUS    252
7  Dane van Niekerk   SA    243
8     Sophie Devine   NZ    242
9       Amelia Kerr   NZ    236


## Top 10 ODI player Womens

In [242]:
get_player_ranking("https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting")

              PLAYER TEAM RATING
0     Tammy Beaumont  ENG    765
1        Lizelle Lee   SA    758
2       Alyssa Healy  AUS    756
3    Stafanie Taylor   WI    746
4        Meg Lanning  AUS    723
5  Amy Satterthwaite   NZ    715
6    Smriti Mandhana  IND    710
7        Mithali Raj  IND    709
8     Natalie Sciver  ENG    685
9    Laura Wolvaardt   SA    683


# Amazon.com

In [453]:
def get_amazon_product_details(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(url,headers=headers)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent, 'html.parser')
    print(soup.find_all("li",{"class":"a-disabled"}))
    total_pages =  int(soup.find_all("li",{"class":"a-disabled"})[-1].text)
    
    df = pd.DataFrame({"Product_Name":[],"Product_Price":[],"Avg_Rating":[],"Image_URL":[]})
    
    for page in tqdm(range(1,401)):
        name, price,  image = [], [], []
        #rating = []
        time.sleep(random.choice(list(range(0,5))))
        try:
            page_url = url+"&page="+str(page)
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            r = requests.get(page_url,headers=headers)
            htmlContent = r.content
            soup = BeautifulSoup(htmlContent, 'html.parser')
            for i in soup.find_all("span",{"class":"a-size-medium a-color-base a-text-normal"}):
                if i.text != None:
                    name.append(i.text)
                else:
                    name.append("None")
            for i in soup.find_all("span",{"class":"a-price-whole"}):
                if i.text != None:
                    price.append(i.text.replace(",",""))
                else:
                    price.append("None")
            """
            for i in soup.find_all("div",{"class":"a-row a-size-small"}):
                if i.find("i") != None:
                    rating.append(i.find("i").text.split(" ")[0])
                else:
                    rating.append("None")
            """
            for i in soup.find_all("img",{"class":"s-image"}):
                if i["src"] != None:
                    image.append(i["src"])
                else:
                    image.append("None")
            df = df.append(pd.DataFrame({"Product_Name":name,"Product_Price":price[-len(name):],"Image_URL":image}))
            #df = df.append(pd.DataFrame({"Product_Name":name,"Product_Price":price[-len(name):],"Avg_Rating":rating,"Image_URL":image}))
        except Exception as e:
            print(e)
    
    df.to_csv("Amazon_mobile_list_under_20000.csv",index=False)

In [474]:
get_amazon_product_details("https://www.amazon.in/s?k=mobile+phones+under+20000&i=electronics&rh=n%3A976419031%2Cn%3A1389401031%2Cn%3A1389432031%2Cn%3A1805560031&dc&qid=1620922649")

[]


IndexError: list index out of range